## DB funcs for nlp

In [8]:
%run ppa_init.ipynb
%run ppa_nlpfuncs.ipynb
work=random.choice(digiworks)
work.title, work.pub_date

('Vowel Alliteration in Modern Poetry.', 1915)

### SQL-style: TotalCount and WordCount

In [9]:
def store_counts(self, countd=None, **opts):
    if not countd: countd = self.calc_counts()
    total = sum(countd.values())
    
    with timer('init workcount'):
        totalcount, totalcount_created = TotalCount.objects.get_or_create(
            work=self, count=total
        )

    ## store words
    with timer('gathering objects to store'):
        objs = [
            WordCount(
                work=self,
                word=word,
                count=countd[word]
            )
            for word in countd
            if word and countd[word]
        ]

    with timer('bulk storing'):
        res = WordCount.objects.bulk_create(objs, ignore_conflicts=True, batch_size=500)
    
    return res

DigitizedWork.store_counts = store_counts

In [10]:
wordcounts = objs = work.store_counts()
len(objs), objs[0], objs[0].__dict__

init workcount ... 0.0s
gathering objects to store ... 0.3s
bulk storing ... 0.8s


(27326,
 WordCount(Work(1264), "the", 13333),
 {'_state': <django.db.models.base.ModelState at 0x1457becd0>,
  'id': None,
  'work_id': 1264,
  'word': 'the',
  'count': 13333})

In [12]:
qset=WordCount.objects.filter(word='ballad')
qset

<QuerySet [WordCount(Work(4191), "ballad", 1), WordCount(Work(2057), "ballad", 1), WordCount(Work(2818), "ballad", 1), WordCount(Work(6327), "ballad", 7), WordCount(Work(4510), "ballad", 1), WordCount(Work(490), "ballad", 2), WordCount(Work(493), "ballad", 1), WordCount(Work(3176), "ballad", 9), WordCount(Work(500), "ballad", 1), WordCount(Work(4679), "ballad", 1), WordCount(Work(2684), "ballad", 1), WordCount(Work(2812), "ballad", 1), WordCount(Work(2990), "ballad", 1), WordCount(Work(2813), "ballad", 1), WordCount(Work(2348), "ballad", 1), WordCount(Work(2817), "ballad", 1), WordCount(Work(241), "ballad", 1), WordCount(Work(499), "ballad", 1), WordCount(Work(2815), "ballad", 1), WordCount(Work(352), "ballad", 1), '...(remaining elements truncated)...']>

### Storing all counts

In [7]:
if 0: 
    timer.PRINT = False

    pmap_run(
        store_counts,
        digiworks,
        num_proc=8
    )

Mapping store_counts() [x8]: 100%|██████████| 5124/5124 [23:45<00:00,  3.59it/s]  
